In [1]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd
import re
from contextlib import redirect_stdout

In [2]:
file_path = "candidates.csv"
df = pd.read_csv(file_path)

In [3]:
cols_to_convert = ["rPM", "Δ Mag", "Δ SEP", "obs", "sep_last"]
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

filtered = df[
    ((df["rPM"] < 0.35) | (df["rPM"].isna()))
].copy()

filtered = filtered.sort_values(
    by=["rPM", "Δ Mag", "Δ SEP", "obs"],
    ascending=[True, True, True, False]
)

# Output to CSV
output_path = "best_physical_double_star_candidates_sorted.csv"
filtered.to_csv(output_path, index=False)

print(f"{len(filtered)} best candidates saved to: {output_path}")


177 best candidates saved to: best_physical_double_star_candidates_sorted.csv


In [4]:
def parse_coord(coord_str):
    """
    Parses coord_arcsec_2000 string into an astropy SkyCoord object.
    """
    # Split on first + or -
    match = re.match(r"^([0-9.]+)([+-][0-9.]+)$", coord_str)
    if not match:
        raise ValueError(f"Invalid coordinate string: {coord_str}")

    ra_raw, dec_raw = match.groups()

    # Convert RA from hhmmss.ss to hms
    ra_h = ra_raw[0:2]
    ra_m = ra_raw[2:4]
    ra_s = ra_raw[4:]

    # Convert Dec from ddmmss.s to dms
    dec_sign = dec_raw[0]
    dec_d = dec_raw[1:3]
    dec_m = dec_raw[3:5]
    dec_s = dec_raw[5:]

    ra_str = f"{ra_h}h{ra_m}m{ra_s}s"
    dec_str = f"{dec_sign}{dec_d}d{dec_m}m{dec_s}s"

    return SkyCoord(ra=ra_str, dec=dec_str, frame="icrs")

In [5]:
df = pd.read_csv("best_physical_double_star_candidates_sorted.csv")

# Query Gaia DR3 for each coordinate
Vizier.columns = ["Source", "Plx", "e_Plx", "Gmag", "pmRA", "pmDE"]
with open("output.txt", "w", encoding="utf-8") as f:
    with redirect_stdout(f):
        for i, row in df.iterrows():
            try:
                coord = parse_coord(row["coord_arcsec_2000"])
                result = Vizier.query_region(coord, radius=2 * u.arcmin, catalog="I/355/gaiadr3", column_filters={'Gmag': '<14'})
        
                if result:
                    print(f"\n===== WDS ID: {row["wds_id"]} =====")
                    print(f"\nObservations: {row["obs"]} | Last: {row["last"]} | First: {row["first"]} ({row["obs"]/row["Years of Obs"]} per year)")
                    if pd.isna(row["rPM"]):
                        print(f"Not available")
                    else:
                        print(f"\nrPM: {row["rPM"]} | Pri P.M.: {row["PM 1"]} | Sec P.M.: {row["PM 2"]}")
                        if row["rPM"] <= 3:
                            print(f"Proper motion indicates physical")
                    print(f"Pri P.M. R.A. {row["pm1_ra"]} | Pri P.M. Dec. {row["pm1_dec"]} | Sec P.M. R.A. {row["pm2_ra"]} | Sec P.M. Dec. {row["pm2_dec"]}")
                    print(f"\nPri mag: {row["mag1"]} | Sec mag: {row["mag2"]} | Δ Mag: {row["Δ Mag"]}")
                    print(f"P.A. first: {row["pa_first"]} | P.A. last: {row["pa_last"]} | Δ P.A.: {row["Δ P.A."]}")
                    print(f"Sep first: {row["sep_first"]} | Sep last: {row["sep_last"]} | Δ Sep: {row["Δ SEP"]}\n")
                    
                    print(f"-- GAIA DR3/I/355 (region around {row["coord_arcsec_2000"]} within 2 arcmin) --\n")
                    # Find pair with smallest parallax difference
                    min_delta_parallax = float("inf")
                    best_pair = (None, None)
        
                    plx_values = result[0]["Plx"].data  # Access parallax values as array
                    e_values = result[0]["e_Plx"].data  # Access error values as array
        
                    for j in range(len(plx_values)):
                        for k in range(j + 1, len(plx_values)):
                            delta = abs(plx_values[j] - plx_values[k])
                            if delta < min_delta_parallax:
                                min_delta_parallax = delta
                                best_pair = (j, k)
                    
                    if best_pair[0] is not None and best_pair[1] is not None:
                        p1 = result[0][best_pair[0]]
                        p2 = result[0][best_pair[1]]
                        
                        if (p1["Gmag"] > p2["Gmag"]):
                            old_p2 = p2
                            p2 = p1
                            p1 = old_p2
                    
                        # Convert each parallax to distance in parsecs
                        d1_pc = 1000 / (p1["Plx"] + p1["e_Plx"])
                        d2_pc = 1000 / (p2["Plx"] - p2["e_Plx"])
                    
                        # Compute radial (line-of-sight) separation in parsecs
                        radial_sep_pc = abs(d1_pc - d2_pc)
                    
                        # Convert to AU and light-years
                        radial_sep_au = radial_sep_pc * u.pc.to(u.au)
                        radial_sep_ly = radial_sep_pc * u.pc.to(u.lyr)
                    
                        print(f"Min radial separation: {radial_sep_pc:.6f} pc | {radial_sep_au:,.0f} AU | {radial_sep_ly:.4f} ly")
                        if radial_sep_au <= 10000:
                            print(f"Radial seperation indicates physical")
                        print(f"Based on Gaia Sources: {p1['Source']}, {p2['Source']}\n\n")
                    else:
                        print("Not enough Gaia sources with valid parallax to compute separation.")
                        
                    print(result[0])
            except Exception as e:
                print(f"Error for {row['coord_arcsec_2000']}: {e}")